In [1]:
import sys
import os

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

print("Path configurado para importar módulos de 'chemai':")
print(ROOT_DIR)


from proxy import configure_proxy
configure_proxy(ROOT_DIR)


Path configurado para importar módulos de 'chemai':
C:\Users\f0pi\git\viscosidade-ai
Proxy configurado.


In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import  SelectKBest, f_regression, VarianceThreshold

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import TruncatedSVD

import os
import joblib
from joblib import dump, load

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import optuna

import torch
import random
import onnxmltools

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

from chemai.loader import DipprDatasetLoader
from chemai.chem_featurizer import ChemFeaturizer
from chemai.chemberta_featurizer import ChemBERTaFeaturizer
from chemai.train import train_test_split



In [51]:
GLOBAL_SEED = 13
np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
os.environ["PYTHONHASHSEED"] = str(GLOBAL_SEED)

In [4]:
data_loader = DipprDatasetLoader(data_dir='../data/nist_dippr_data')
data_loader.prepare()

pure = data_loader.get_pure()
mix  = data_loader.get_mix()

pure_train, pure_dev = train_test_split(
    smiles1=pure['train']["MOL"],
    T=pure['train']["T"],
    logV=pure['train']["logV"]
)
pure_test = pure['test']  # conjunto final de teste

mix_train, mix_dev = train_test_split(
    smiles1=mix['train']["MOL_1"],
    smiles2=mix['train']["MOL_2"],
    MolFrac_1=mix['train']["MolFrac_1"],
    T=mix['train']["T"],
    logV=mix['train']["logV"]
)
mix_test = mix['test']  # conjunto final de teste

print(f"Puro: train={len(pure_train['smiles'])}, dev={len(pure_dev['smiles'])}, test={pure_test.shape[0]}")
print(f"Mix: train={len(mix_train['smiles_1'])}, dev={len(mix_dev['smiles_1'])}, test={mix_test.shape[0]}")


Puro: train=5268, dev=1450, test=885
Mix: train=20635, dev=5254, test=5585


In [53]:
chem = ChemFeaturizer()
chemberta = ChemBERTaFeaturizer(device='cuda' if torch.cuda.is_available() else 'cpu')

# PUROS
df_train_pure = pd.DataFrame({'MOL': pure_train['smiles'], 'T': pure_train['T'], 'logV': pure_train['logV']})
df_dev_pure   = pd.DataFrame({'MOL': pure_dev['smiles'],   'T': pure_dev['T'],   'logV': pure_dev['logV']})
df_test_pure  = pure_test.copy()

feat_train_pure_chem = chem.featurize_pure(df_train_pure)
feat_dev_pure_chem   = chem.featurize_pure(df_dev_pure)
feat_test_pure_chem  = chem.featurize_pure(df_test_pure)

feat_train_pure_bert = chemberta.featurize_pure(df_train_pure)
feat_dev_pure_bert   = chemberta.featurize_pure(df_dev_pure)
feat_test_pure_bert  = chemberta.featurize_pure(df_test_pure)

# MISTURAS
df_train_mix = pd.DataFrame({
    'MOL_1': mix_train['smiles_1'], 'MOL_2': mix_train['smiles_2'],
    'MolFrac_1': mix_train['MolFrac_1'], 'T': mix_train['T'], 'logV': mix_train['logV']})
df_dev_mix = pd.DataFrame({
    'MOL_1': mix_dev['smiles_1'], 'MOL_2': mix_dev['smiles_2'],
    'MolFrac_1': mix_dev['MolFrac_1'], 'T': mix_dev['T'], 'logV': mix_dev['logV']})
df_test_mix = mix_test.copy()

feat_train_mix_chem = chem.featurize_mix_parallel(df_train_mix)
feat_dev_mix_chem   = chem.featurize_mix_parallel(df_dev_mix)
feat_test_mix_chem  = chem.featurize_mix_parallel(df_test_mix)

feat_train_mix_bert = chemberta.featurize_mix(df_train_mix)
feat_dev_mix_bert   = chemberta.featurize_mix(df_dev_mix)
feat_test_mix_bert  = chemberta.featurize_mix(df_test_mix)


Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-77M-MTR and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
def build_pipeline(model, X_sample, embedding_features, k_features=50):
    if not embedding_features:
        cols = list(X_sample.columns)
        ignore = [c for c in cols if 'has_' in c or c == 'frac']
        floats = [c for c in cols if np.issubdtype(X_sample[c].dtype, np.floating) and c not in ignore]
        ints = [c for c in cols if np.issubdtype(X_sample[c].dtype, np.integer) and c not in ignore]


        transformers = []
        if floats: transformers.append(('float_scaler', StandardScaler(), floats))
        if ints: transformers.append(('int_scaler', MinMaxScaler(), ints))
        if ignore: transformers.append(('skip', 'passthrough', ignore))

        col_transform = ColumnTransformer(transformers)

        pipeline = Pipeline([
            ('scaler', col_transform),
            ('imputer', SimpleImputer(strategy='median')),
            ('kbest', SelectKBest(score_func=f_regression, k=k_features)),
            ('model', model)
        ])
        
        return pipeline
    
    pipeline = Pipeline([
        # ('svd', TruncatedSVD(n_components=100, random_state=GLOBAL_SEED)),
        ('selector', VarianceThreshold(0.0)),
        ('model', model)
    ])
    return pipeline 


In [46]:
def optuna_objective(trial, model_name, X_train, y_train, X_dev, y_dev, embedding_features):
    if model_name == 'SVR':
        params = {
            'C': trial.suggest_float('C', 1e-2, 1e2, log=True),
            'kernel': trial.suggest_categorical('kernel', ['rbf', 'linear']),
            'gamma': trial.suggest_categorical('gamma', ['scale', 'auto'])
        }
        model = SVR(**params)

    elif model_name == 'XGB':
        params = {
            'n_estimators': trial.suggest_categorical('n_estimators', [200, 400, 800]),
            'max_depth': trial.suggest_int('max_depth', 3, 8),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.7, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
            'random_state': GLOBAL_SEED
        }
        model = XGBRegressor(**params, n_jobs=-1)

    elif model_name == 'LGB':
        params = {
            'n_estimators': trial.suggest_categorical('n_estimators', [200, 400, 800]),
            'max_depth': trial.suggest_int('max_depth', 3, 8),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('subsample', 0.7, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
            'random_state': GLOBAL_SEED
        }
        model = LGBMRegressor(**params, n_jobs=-1, verbose=-1)

    elif model_name == 'RF':
        params = {
            'n_estimators': trial.suggest_categorical('n_estimators', [200, 400, 800]),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 5),
            'random_state': GLOBAL_SEED,
        }
        model = RandomForestRegressor(**params,  n_jobs=-1)

    k_features = trial.suggest_int('k_features', 50, X_train.shape[1])
    pipeline = build_pipeline(model, X_train, embedding_features, k_features)

    pipeline.fit(X_train, y_train)

    y_pred_dev = pipeline.predict(X_dev)
    r2_dev = r2_score(y_dev, y_pred_dev)
    rmse_dev = np.sqrt(mean_squared_error(y_dev, y_pred_dev))

    trial.set_user_attr('rmse_dev', rmse_dev)
    trial.set_user_attr('k_features', k_features)
    trial.set_user_attr('params', params)

    return r2_dev


In [47]:
def export_pipeline_to_onnx(pipeline, columns, feat_name, model_name):
    initial_types = [(col, FloatTensorType([None, 1])) for col in columns]
    
    if model_name not in {'XGB', 'LGB'}:
        onnx_path = f"../models/sklearn/{feat_name}_{model_name}_optuna_pipeline.onnx"

        onnx_model = convert_sklearn(pipeline, initial_types=initial_types)

        with open(onnx_path, "wb") as f:
            f.write(onnx_model.SerializeToString())
    else:
        preprocess = Pipeline(pipeline.steps[:-1])
        model = pipeline.named_steps["model"]


        preprocess_path = f"../models/sklearn/{feat_name}_{model_name}_optuna_preprocess.onnx"

        onnx_pre = convert_sklearn(preprocess, initial_types=initial_types)
        with open(preprocess_path, "wb") as f:
            f.write(onnx_pre.SerializeToString())

        model_path = f"../models/sklearn/{feat_name}_{model_name}_optuna_model.onnx"
        if model_name == 'XGB':
            xgb_model.get_booster().save_model(model_path)
        elif model_name == 'LGB':
            onnx_model = onnxmltools.convert_lightgbm(
            model, initial_types=[('float_input', FloatTensorType([None, len(columns)]))])
            onnxmltools.utils.save_model(onnx_model, model_path)

In [48]:
n_trials = 100
results = []
os.makedirs("../models/sklearn", exist_ok=True)

datasets = {
    'ChemFeaturizer_Puro': (feat_train_pure_chem, feat_dev_pure_chem, feat_test_pure_chem),
    'ChemFeaturizer_Mix':  (feat_train_mix_chem,  feat_dev_mix_chem,  feat_test_mix_chem),
    'ChemBERTa_Puro': (feat_train_pure_bert, feat_dev_pure_bert, feat_test_pure_bert),
    'ChemBERTa_Mix': (feat_train_mix_bert, feat_dev_mix_bert, feat_test_mix_bert)
}

sampler = optuna.samplers.TPESampler(seed=GLOBAL_SEED)

for feat_name, (df_train, df_dev, df_test) in datasets.items():
    X_train, y_train = df_train.drop(columns=['logV']), df_train['logV']
    X_dev, y_dev = df_dev.drop(columns=['logV']), df_dev['logV']
    X_test, y_test = df_test.drop(columns=['logV']), df_test['logV']

    for model_name in  ['RF', 'SVR', 'XGB', 'LGB']:
        if model_name in ['RF', 'SVR'] and feat_name == 'ChemBERTa_Mix':
            continue
        print(f"\n Optuna - {model_name} ({feat_name})")
        study = optuna.create_study(direction='maximize', sampler=sampler)
        study.optimize(lambda trial: optuna_objective(
            trial, model_name, X_train, y_train, X_dev, y_dev, feat_name.startswith('ChemBERTa')),
                       n_trials=n_trials, show_progress_bar=True)

        best_trial = study.best_trial
        best_params = best_trial.user_attrs['params']

        X_full = pd.concat([X_train, X_dev], axis=0).reset_index(drop=True)
        y_full = pd.concat([y_train, y_dev], axis=0).reset_index(drop=True)

        if model_name == 'SVR':
            model_final = SVR(**best_params)
        elif model_name == 'XGB':
            model_final = XGBRegressor(**best_params)
        elif model_name == 'RF':
            model_final = RandomForestRegressor(**best_params)
        elif model_name == 'LGB':
            model_final = LGBMRegressor(**best_params)

        k_features = best_trial.user_attrs['k_features']
        print(f"Melhor número de features selecionadas: {k_features}")

        full_pipeline = build_pipeline(model_final, X_full, feat_name.startswith('ChemBERTa'), k_features)
        full_pipeline.fit(X_full, y_full)
        y_pred_test = full_pipeline.predict(X_test)

        r2 = r2_score(y_test, y_pred_test)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

        filename = f"../models/sklearn/{feat_name}_{model_name}_optuna_pipeline.joblib"
        dump(full_pipeline, filename)

        export_pipeline_to_onnx(full_pipeline, X_train.columns, feat_name, model_name)

        print(f" Pipeline salva: {filename}")
        print(f" [TEST] R²={r2:.4f}, RMSE={rmse:.4f}")

        results.append({
            'featurizer': feat_name,
            'model': model_name,
            'r2_test': r2,
            'rmse_test': rmse,
            'params': best_params,
            'file': filename
        })


[I 2025-12-16 03:52:36,120] A new study created in memory with name: no-name-859a3a29-3ffc-4191-b9cb-8a2aecaf471d



 Optuna - LGB (ChemBERTa_Mix)


  0%|          | 0/100 [00:00<?, ?it/s]

C:\Users\f0pi\AppData\Local\miniforge3\envs\chemai\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[I 2025-12-16 03:53:02,414] Trial 0 finished with value: 0.9130334989034612 and parameters: {'n_estimators': 800, 'max_depth': 8, 'learning_rate': 0.2733063849296914, 'subsample': 0.8360347742251937, 'colsample_bytree': 0.8827127388283833, 'k_features': 609}. Best is trial 0 with value: 0.9130334989034612.


...

[I 2025-12-16 04:34:08,015] Trial 98 finished with value: 0.9307027321851657 and parameters: {'n_estimators': 800, 'max_depth': 7, 'learning_rate': 0.12141309208225555, 'subsample': 0.761212665237573, 'colsample_bytree': 0.763997048861721, 'k_features': 167}. Best is trial 86 with value: 0.9377591282274104.


C:\Users\f0pi\AppData\Local\miniforge3\envs\chemai\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[I 2025-12-16 04:34:36,458] Trial 99 finished with value: 0.9276864001126544 and parameters: {'n_estimators': 800, 'max_depth': 7, 'learning_rate': 0.1493828511862724, 'subsample': 0.7128211365883007, 'colsample_bytree': 0.7971091142624482, 'k_features': 274}. Best is trial 86 with value: 0.9377591282274104.
Melhor número de features selecionadas: 341


C:\Users\f0pi\AppData\Local\miniforge3\envs\chemai\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


 Pipeline salva: ../models/sklearn/ChemBERTa_Mix_LGB_optuna_pipeline.joblib
 [TEST] R²=0.8400, RMSE=0.1436


In [79]:
def load_models_and_metrics(folder="../models/sklearn"):
    models = [f for f in os.listdir(folder) if f.endswith(".joblib")]
    results = []
    for file_name in models:
        model_name = file_name.replace("_optuna_pipeline.joblib", "")
        model = load(os.path.join(folder, file_name))
        if "ChemBERTa_Mix" in model_name:
            df_test = feat_test_mix_bert
            featurizer = "ChemBERTa"
            data_type = "Mix"
        elif "ChemFeaturizer_Mix" in model_name:
            df_test = feat_test_mix_chem
            featurizer = "ChemFeaturizer"
            data_type = "Mix"
        elif "ChemBERTa_Puro" in model_name:
            df_test = feat_test_pure_bert
            featurizer = "ChemBERTa"
            data_type = "Puro"
        elif "ChemFeaturizer_Puro" in model_name:
            df_test = feat_test_pure_chem
            featurizer = "ChemFeaturizer"
            data_type = "Puro"
        else:
            continue

        if "XGB" in model_name:
            model_type = "XGB"
        elif "LGB" in model_name:
            model_type = "LGB"
        elif "RF" in model_name:
            model_type = "RF"
        elif "SVR" in model_name:
            model_type = "SVR"
        else:
            model_type = "Unknown"

        X_test = df_test.drop(columns=["logV"])
        y_test = df_test["logV"]
        y_pred = model.predict(X_test)

        if data_type == 'Mix':
            mol1 = [col for col in X_test.columns if col.startswith('mol1')]
            mol2 = [col for col in X_test.columns if col.startswith('mol2')]
            inv_mols = mol2 + mol1 + ['frac'] + ['T']
            X_test_inv = X_test[inv_mols].copy()
            X_test_inv.columns = X_test.columns
            y_pred_inv = model.predict(X_test_inv)
            y_pred = 0.5 * y_pred + 0.5 * y_pred_inv

        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))

        results.append({
            "Featurizer": featurizer,
            "Type": data_type,
            "Model": model_type,
            "R2": r2,
            "RMSE": rmse
        })

    df_results = pd.DataFrame(results)
    df_results = df_results.sort_values(by=["Type", "Featurizer", "Model"]).reset_index(drop=True)
    return df_results

metrics_df = load_models_and_metrics()
metrics_df


C:\Users\f0pi\AppData\Local\miniforge3\envs\chemai\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\f0pi\AppData\Local\miniforge3\envs\chemai\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


,Featurizer,Type,Model,R2,RMSE
0,ChemBERTa,Mix,LGB,0.678780,0.203529
1,ChemBERTa,Mix,XGB,0.695918,0.198025
2,ChemFeaturizer,Mix,XGB,0.719597,0.190159
3,ChemBERTa,Puro,RF,0.890496,0.141315
4,ChemBERTa,Puro,SVR,0.944337,0.100753
5,ChemBERTa,Puro,XGB,0.938045,0.106295
6,ChemFeaturizer,Puro,RF,0.972323,0.071045
7,ChemFeaturizer,Puro,SVR,0.959024,0.086445
8,ChemFeaturizer,Puro,XGB,0.953514,0.092074
